In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))

In [3]:
from finetune_flat_classifiers import * 
from sklearn.preprocessing import LabelEncoder
import pyarrow.parquet as pq
import utils, os

In [7]:
settings = utils.load_settings(path="settings_sample.json")
features_dir = settings["features_dir"]
cleaned_data_dir = settings["cleaned_data_dir"]
reports_dir = settings["reports_dir"]
trials = settings["optuna_trials"]
trials = 1

In [3]:
X_table = pq.read_table(os.path.join(features_dir, "murugaiah_features.parquet"))
X_murugaiah = X_table.to_pandas().to_numpy()

X_table = pq.read_table(os.path.join(features_dir, "kmer_features_6.parquet"))
X_kmer_6 = X_table.to_pandas().to_numpy()

X_table = pq.read_table(os.path.join(features_dir, "kmer_features_5.parquet"))
X_kmer_5 = X_table.to_pandas().to_numpy()

X_table = pq.read_table(os.path.join(features_dir, "fcgr_features.parquet"))
X_fcgr = X_table.to_pandas().to_numpy()

In [4]:
features_dict = {
    "murugaiah": X_murugaiah,
    # "kmer_6": X_kmer_6,
    # "kmer_5": X_kmer_5,
    # "fcgr": X_fcgr
}

In [8]:
y_table = pq.read_table(os.path.join(cleaned_data_dir, "flat_labels.parquet"))
y = y_table.to_pandas().to_numpy()

le = LabelEncoder()
y = le.fit_transform(y.flatten())

In [22]:
import numpy as np
n_splits = 5

classes, class_counts = np.unique(y, return_counts=True)

valid_classes = classes[class_counts >= n_splits]
valid_indices = np.isin(y, valid_classes)

print(f"{len(y)} samples from {len(classes)} classes in the dataset.")
print(f"{len(valid_classes)} classes have at least {n_splits} instances.")
print(f"{len(y[valid_indices])} instances are in these classes.")

42037 samples from 580 classes in the dataset.
287 classes have at least 5 instances.
41510 instances are in these classes.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_murugaiah, y, test_size=0.2, random_state=42)
    
xgb = XGBClassifier(
    learning_rate=0.1,
    n_estimators=10,
    max_depth=20,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    min_child_weight=1,
    objective='multi:softmax',
    random_state=42,
    verbosity=3,
    n_jobs=-1,
)

xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10)

## Random Forest

In [ ]:
for name, features in features_dict.items():
    finetune_rf(features, y, name, n_trials=trials, reports_dir=reports_dir)

## XGBoost

In [ ]:
for name, features in features_dict.items():
    finetune_xgb(features, y, name, n_trials=trials, reports_dir=reports_dir)

## LightGBM

In [ ]:
for name, features in features_dict.items():
    finetune_lgbm(features, y, name, n_trials=trials, reports_dir=reports_dir)

## CatBoost

In [ ]:
for name, features in features_dict.items():
    finetune_cb(features, y, name, n_trials=trials, reports_dir=reports_dir)